In [1]:
from glob import glob
import numpy as np
import os
from PIL import Image
data_dir = '../data'

In [2]:
labels = os.listdir(data_dir)
print(labels)

['axes', 'boots', 'carabiners', 'crampons', 'gloves', 'hardshell_jackets', 'harnesses', 'helmets', 'insulated_jackets', 'pulleys', 'rope', 'tents']


In [3]:
#configuration
img_width, img_height = 64, 64
batch_size=4

In [4]:
X, y = [], []
for label in labels:
    files = (os.listdir(os.path.join(data_dir,label)))
    for file in files:
        img = Image.open(os.path.join(data_dir, label,file)).convert('RGB')
        width, height = img.size
        img = img.resize((img_width, img_height), Image.ANTIALIAS)
        data = np.asarray(img)
        X.append(data)
        y.append(labels.index(label))

In [5]:
import random
c = list(zip(X, y))
random.shuffle(c)
X, y = zip(*c)

In [6]:
from keras.utils import np_utils
y = np_utils.to_categorical(y) 

Using TensorFlow backend.


In [8]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
import random

# dimensions of our images.
num_classes=len(labels)
X_train = np.array(X)
y_train = np.array(y)
X_train = X_train.reshape(X_train.shape[0], img_width, img_height, 3)
input_shape = (img_width, img_height, 3)


datagen = ImageDataGenerator(
    rescale=1./255,
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit_generator(datagen.flow(np.array(X_train), np.array(y_train), batch_size=batch_size),steps_per_epoch=len(X_train), epochs=10, validation_split=0.2, verbose=1)

TypeError: fit_generator() got an unexpected keyword argument 'validation_split'

In [16]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K

datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    preprocessing_function = )

# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
datagen.fit(X_train)


# create the base pre-trained model
base_model = ResNet50(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(num_classes, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

# fits the model on batches with real-time data augmentation:
model.fit_generator(datagen.flow(X_train, y_train, batch_size=32),steps_per_epoch=len(x_train) / 32, epochs=epochs)

94658560/94653016 [==============================] - 72s 1us/step


c:\users\lahulsta\appdata\local\continuum\anaconda3\envs\openhack\lib\site-packages\ipykernel_launcher.py:29: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 1697 samples, validate on 425 samples
Epoch 1/10
1697/1697 [==============================] - 24s 14ms/step - loss: 1.4026 - acc: 0.6187 - val_loss: 2.8913 - val_acc: 0.0682
Epoch 2/10
1697/1697 [==============================] - 26s 15ms/step - loss: 0.8534 - acc: 0.7725 - val_loss: 3.7918 - val_acc: 0.0847
Epoch 3/10
1224/1697 [====================>.........] - ETA: 7s - loss: 0.7752 - acc: 0.8039

KeyboardInterrupt: 